In [1]:
import logging
logging.basicConfig(level=logging.INFO)

Throughout this section, we'll be building on the following Cicada program, which adds two numbers using additive secret sharing:

In [2]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

SocketCommunicator.run(fn=main, world_size=3);

Note that we're using a very small field (`order=127`) and reduced precision encoding (`precision=2`) to keep the output values manageable.

In [3]:
from cicada import transcript

def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)
    
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

In [4]:
def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)
    
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's share some secrets!
Player 2: Let's add some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 2: Let's reveal the results!
Player 1: Let's reveal the results!
Player 0: Let's reveal the results!


In [5]:
def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter())
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.ShowReceivedMessages())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)

    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: --> 1 PRZS 9058657319882721885
Player 1: --> 2 PRZS 5078680220890093857
Player 2: --> 0 PRZS 6012564425437213046
Player 1: <-- 0 PRZS 9058657319882721885
Player 0: <-- 2 PRZS 6012564425437213046
Player 2: <-- 1 PRZS 5078680220890093857
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 2: Let's add some secrets!
Player 1: Let's reveal the results!
Player 0: Let's reveal the results!
Player 2: Let's reveal the results!
Player 2: --> 0 GATHER 126
Player 0: --> 0 GATHER 100
Player 1: --> 0 GATHER 48
Player 0: <-- 0 GATHER 100
Player 1: --> 1 GATHER 48
Player 2: --> 1 GATHER 126
Player 0: <-- 2 GATHER 126
Player 1: <-- 1 GATHER 48
Player 2: --> 2 GATHER 126
Player 0: <-- 1 GATHER 48
Player 1: <-- 2 GATHER 126
Player 2: <-- 2 GATHER 

In [6]:
def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter(msgfmt="{message.comm.rank} {message.dir} {message.other} {message.tag} {message.payload}"))
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.ShowReceivedMessages())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)

    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
0 > 1 PRZS 1784362653799380138
1 > 2 PRZS 536884848533061514
2 > 0 PRZS 8487418460553309579
1 < 0 PRZS 1784362653799380138
0 < 2 PRZS 8487418460553309579
2 < 1 PRZS 536884848533061514
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 0: Let's reveal the results!
Player 2: Let's reveal the results!
Player 1: Let's reveal the results!
0 > 0 GATHER 93
2 > 0 GATHER 103
1 > 0 GATHER 78
2 > 1 GATHER 103
0 < 0 GATHER 93
1 > 1 GATHER 78
0 < 2 GATHER 103
2 > 2 GATHER 103
1 < 1 GATHER 78
0 < 1 GATHER 78
1 < 2 GATHER 103
2 < 2 GATHER 103
0 > 1 GATHER 93
0 > 2 GATHER 93
1 < 0 GATHER 93
2 < 0 GATHER 93
1 > 2 GATHER 78
2 < 1 GATHER 78


Note from the log outputs that an entry is generated both when a message is sent *and* when it's received, and that the rank of the  player logging the event is always on the left, whether they are the sender or the receiver.  With the arrows indicating which direction the message is travelling, this is intuitive for a person to understand, but it can make programmatically parsing the transcript needlessly difficult.  As an alternative, you can adjust the format to always put the sender on the left and the recipient on the right, regardless of which player is logging the event:

In [7]:
def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter(msgfmt="send from {message.src} to {message.dst} {message.tag} {message.payload}"))
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.ShowReceivedMessages())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)

    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

Player 0: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
send from 0 to 1 PRZS 4521381316237156826
send from 2 to 0 PRZS 9107140275150719362
send from 1 to 2 PRZS 3839543892411096429
send from 2 to 0 PRZS 9107140275150719362
send from 1 to 2 PRZS 3839543892411096429
send from 0 to 1 PRZS 4521381316237156826
Player 0: Let's share some secrets!
Player 1: Let's share some secrets!
Player 2: Let's share some secrets!
Player 2: Let's add some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
send from 0 to 0 GATHER 91
send from 1 to 0 GATHER 50
send from 2 to 0 GATHER 6
send from 1 to 1 GATHER 50
send from 2 to 1 GATHER 6
send from 0 to 0 GATHER 91
send from 1 to 0 GATHER 50
send from 2 to 2 GATHER 6
send from 1 to 1 GATHER 50
send from 2 to 0 GATHER 6
send from 2 to 1 GATHER 6
send from 2 to 2 G

In [8]:
def main(communicator):
    handler = logging.StreamHandler()
    handler.setFormatter(transcript.Formatter(msgfmt="send from {message.src} to {message.dst} {message.tag} {message.payload}"))
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.HideAllFunctions())
    transcript.logger.addHandler(handler)

    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
send from 0 to 1 PRZS 2797978244959157213
send from 1 to 2 PRZS 9109704083374206616
send from 2 to 0 PRZS 4948755585000050858
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 2: Let's add some secrets!
Player 1: Let's add some secrets!
Player 0: Let's add some secrets!
Player 2: Let's reveal the results!
Player 1: Let's reveal the results!
Player 0: Let's reveal the results!
send from 1 to 0 GATHER 0
send from 0 to 0 GATHER 40
send from 2 to 0 GATHER 107
send from 1 to 1 GATHER 0
send from 2 to 1 GATHER 107
send from 0 to 1 GATHER 40
send from 2 to 2 GATHER 107
send from 0 to 2 GATHER 40
send from 1 to 2 GATHER 0


## Tracing Function Calls

In [9]:
def main(communicator):
    handler = logging.FileHandler(f"player-{communicator.rank}.log", mode="w")
    handler.setFormatter(transcript.Formatter())
    handler.addFilter(transcript.ShowSentMessages())
    handler.addFilter(transcript.ShowReceivedMessages())
    handler.addFilter(transcript.HideSpecialFunctions())
    handler.addFilter(transcript.HideInitFunctions())
    handler.addFilter(transcript.HidePrivateFunctions())
    handler.addFilter(transcript.HideDefaultFunctions())
    handler.addFilter(transcript.HideCommunicationFunctions())
    handler.addFilter(transcript.HideSelfArguments())
    handler.addFilter(transcript.MapInlineResults({
        "cicada.arithmetic.Field.inplace_add": "lhs",
        "cicada.arithmetic.Field.inplace_subtract": "lhs",
    }))
    transcript.logger.addHandler(handler)

    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

    handler.close()

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3)

In [10]:
for rank in range(3):
    with open(f"player-{rank}.log", "r") as stream:
        print(stream.read())
        print("\n" * 5)

Player 0: --> 1 PRZS 8405593446333862681
Player 0: <-- 2 PRZS 9153506999110928940
Player 0: Let's share some secrets!
Player 0: AdditiveProtocolSuite.share(src=0, secret=array(2), shape=(), encoding=None)
Player 0:   Field.uniform(size=(), generator=Generator(PCG64) at 0x11526C040)
Player 0:     Field.uniform => 122
Player 0:   Field.uniform(size=(), generator=Generator(PCG64) at 0x11526D2A0)
Player 0:     Field.uniform => 185
Player 0:   Field.inplace_subtract(lhs=array(122, dtype=object), rhs=array(185, dtype=object))
Player 0:     Field.inplace_subtract => 64
Player 0:   FixedPoint.encode(array=array(2), field=cicada.arithmetic.Field(order=127))
Player 0:     FixedPoint.encode => 8
Player 0:   Field.inplace_add(lhs=array(64, dtype=object), rhs=array(8, dtype=object))
Player 0:     Field.inplace_add => 72
Player 0:   AdditiveProtocolSuite.share => cicada.additive.AdditiveArrayShare(storage=72)
Player 0: AdditiveProtocolSuite.share(src=1, secret=array(3), shape=(), encoding=None)
Play